# Постановка задачи
В качестве экзамена вам необходимо решить реальный аналитический кейс, который переложен на данные нашего учебного сайта. 

Вам необходимо выяснить, пользователи с какими устройствами приносят наибольший доход. Под доходом мы подразумеваем ARPU (Average Revenue Per User), то есть отношение выручки к размеру аудитории. Таким образом, результатом решения кейса будет конкретная модель устройства.


# Какие данные у нас есть
1. Выгрузка  с доходом от рекламы сайта в разбивке по моделям устройств 
Данные по рекламным доходам arpu_revenue_data.csv

В этой выгрузке уже просуммирован общий доход (revenue) по каждому устройству за март 2018 года.

Каждое устройство идентифицируется с помощью трех колонок: mobilePhone, mobilePhoneModel и deviceCategory.

2. Данные Яндекс.Метрики по суточной аудитории сайта в разбивке по устройствам
Эти данные вам необходимо получить по API. Номер счетчика учебного сайта для выгрузки данных из Яндекс.Метрики: 21075004

Набор измерений и метрик для получения отчета Яндекс.Метрики 
'ym:s:date', 'ym:s:deviceCategory', 'ym:s:mobilePhone', 'ym:s:mobilePhoneModel', 'ym:s:users'

# Ваша задача
1. Собрать данные о доходах и данные об аудитории в единый отчет

2. Посчитать метрику ARPU по каждому устройству 

ARPU =  revenue/ суммарное количество пользователей

3. Найти модель устройства с наибольшим ARPU

В качестве ответа онлайн-проверка ожидает получить название модели телефона, которой соответствует максимальное значение ARPU. Т. е. если вы получили, например, максимальное значение ARPU для такого устройства:

deviceCategory: Смартфоны

mobilePhone: Sony

mobilePhoneModel: Xperia Z

то в онлайн-проверку надо вставить Xperia Z

# Несколько важных замечаний
1. При расчете суточной аудитории необходимо учитывать только те устройства, которыми за март пользовались не менее 3 пользователей (т. е. сумма суточной аудитории составила 3 и более пользователей)
2. Чтобы получить общую аудиторию за месяц по конкретному устройству, необходимо суммировать количество пользователей за каждый день этому устройству
3. Поскольку задание максимально приближено к "боевым" условиям учитывайте тот факт, что у Метрики есть ограничение по количеству запросов -  и если ваши коллеги его исчерпают, то вы можете не получить выгрузку. В этом случае, попробуйте повторить обращение через некоторое время!
4. Мы создали специальный канал в Slack для ваших вопросов. #exam
5. Мы уверены, что вы успешно справитесь с этой задачей!

In [1]:
! python --version

Python 3.5.2 :: Anaconda 4.2.0 (64-bit)


In [2]:
import pandas as pd

In [16]:
df=pd.read_csv('arpu_revenue_data.csv', sep = '\t')

In [18]:
df.head()

,mobilePhone,mobilePhoneModel,deviceCategory,revenue
0,4Good,Light A104,Смартфоны,"0,987567486"
1,Acer,B1-750,Планшеты,"0,952423812"
2,Acer,E39,Смартфоны,"5,240945719"
3,Acer,T02,Смартфоны,"2,637776834"
4,Acer,T03,Смартфоны,"1,020764495"


In [29]:
df['revenue']=(df['revenue']).apply(lambda x: float(x.replace(',', '.')))

In [33]:
print(type(df['revenue'][0]))
print (df['revenue'][0])

<class 'numpy.float64'>
0.987567486


In [34]:
df.head()

,mobilePhone,mobilePhoneModel,deviceCategory,revenue
0,4Good,Light A104,Смартфоны,0.987567
1,Acer,B1-750,Планшеты,0.952424
2,Acer,E39,Смартфоны,5.240946
3,Acer,T02,Смартфоны,2.637777
4,Acer,T03,Смартфоны,1.020764


In [245]:
df[df['mobilePhone']=='desktop']

,mobilePhone,mobilePhoneModel,deviceCategory,revenue


In [260]:
df[df['deviceCategory']!='Смартфоны'][df['deviceCategory']!='Планшеты']

D:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,mobilePhone,mobilePhoneModel,deviceCategory,revenue
465,None,None,ПК,2667.568827
468,None,None,ТВ,1.299006


In [5]:
import requests

from yaml import load

from pprint import pprint

In [36]:
f = open('config.yaml', 'r')

config = load(f)

token = config['token']

API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'

startDate = '2018-03-01'
endDate = '2018-03-31'

counter = '21075004'

In [69]:
dimensions = ['ym:s:date', 'ym:s:deviceCategory', 'ym:s:mobilePhone', 'ym:s:mobilePhoneModel']

metrics = ['ym:s:users']

metrics_string = ','.join(metrics)


dimensions_string = ','.join(dimensions)

params = {

    'date1': startDate,
    'date2': endDate,
    'id': counter,
    'dimensions': dimensions_string,
    'metrics': metrics_string,
    'oauth_token': token
}

r = requests.get(API_URL, params = params)

data = r.json()
pprint(data)

{'data': [{'dimensions': [{'name': '2018-03-12'},
                          {'icon_id': 'desktop',
                           'icon_type': 'device',
                           'id': 'desktop',
                           'name': 'ПК'},
                          {'icon_id': None,
                           'icon_type': None,
                           'id': None,
                           'name': None},
                          {'icon_id': None, 'icon_type': None, 'name': None}],
           'metrics': [157.0]},
          {'dimensions': [{'name': '2018-03-13'},
                          {'icon_id': 'desktop',
                           'icon_type': 'device',
                           'id': 'desktop',
                           'name': 'ПК'},
                          {'icon_id': None,
                           'icon_type': None,
                           'id': None,
                           'name': None},
                          {'icon_id': None, 'icon_type': None, 'name': None}]

In [51]:
data['data'][0]['metrics'][0]

157.0

In [54]:
len(data['data'])

100

In [58]:
sum_users=0
for i in range (len(data['data'])):
    sum_users+=data['data'][i]['metrics'][0]
sum_users

4227.0

По умолчанию Метрика возвращает первые 100 строк. Добавим параметры limit и offset к запросу.

In [59]:
def report_for_offset_value(offset, n_rows):

    """Выгрузка n_rows строк отчета Метрики для заданного значения offset"""

    

    params = {

        'date1': startDate,

        'date2': endDate,

        'id': counter,

        'dimensions': dimensions_string,

        'metrics': metrics_string,

        'oauth_token': token,

        'offset': offset,

        'limit': n_rows

    }

    

    r = requests.get(API_URL, params = params)

    

    return r.json()

In [62]:
sum_users=0

n_rows=100

offset=1

Started = True

while (len(data['data']) > 0 or Started):
    
    data=report_for_offset_value(offset,n_rows)
    
    Started = False
        
    for line in data['data']:
        
        users=line['metrics'][0]
        
        sum_users+=users
            
        print(sum_users)
        
    offset+=n_rows

157.0
271.0
382.0
491.0
599.0
705.0
808.0
910.0
1007.0
1102.0
1197.0
1291.0
1381.0
1471.0
1557.0
1643.0
1725.0
1807.0
1887.0
1964.0
2041.0
2118.0
2193.0
2268.0
2340.0
2412.0
2481.0
2546.0
2610.0
2672.0
2733.0
2793.0
2852.0
2910.0
2967.0
3024.0
3079.0
3132.0
3184.0
3234.0
3282.0
3329.0
3375.0
3421.0
3465.0
3508.0
3550.0
3587.0
3622.0
3657.0
3691.0
3723.0
3754.0
3785.0
3815.0
3844.0
3872.0
3900.0
3927.0
3952.0
3975.0
3997.0
4008.0
4016.0
4024.0
4032.0
4040.0
4047.0
4054.0
4061.0
4068.0
4075.0
4081.0
4087.0
4093.0
4099.0
4105.0
4111.0
4117.0
4123.0
4129.0
4135.0
4141.0
4147.0
4152.0
4157.0
4162.0
4167.0
4172.0
4177.0
4182.0
4187.0
4192.0
4197.0
4202.0
4207.0
4212.0
4217.0
4222.0
4227.0
4232.0
4237.0
4242.0
4247.0
4252.0
4257.0
4262.0
4266.0
4270.0
4274.0
4278.0
4282.0
4286.0
4290.0
4294.0
4298.0
4302.0
4306.0
4310.0
4314.0
4318.0
4322.0
4326.0
4330.0
4334.0
4338.0
4342.0
4346.0
4350.0
4354.0
4358.0
4362.0
4366.0
4370.0
4374.0
4378.0
4382.0
4386.0
4390.0
4394.0
4398.0
4402.0
4406.0
4410.0


# Проверка 2
(1 возможный балл)
После импорта данных по выручке из файла arpu_revenue_data.csv столбец с выручкой revenue необходимо перевести в тип float.

Какова сумма значений этого столбца? Ответ округлите до ближайшего целого числа.

In [63]:
sum(df['revenue'])

5800.550916065998

# Проверка 3
(1 возможный балл)
Перед тем как совместить выгрузку Яндекс.Метрики и данные по выручке необходимо сделать 2 операции:

1. Сгруппировать выгрузку Яндекс.Метрики по 'deviceCategory', 'mobilePhone', 'mobilePhoneModel', чтобы просуммировать значения по дням (т. к. значения по выручке известны только в сумме за март).

2. После группировки оставить только те строки получившейся таблицы, в которых оказалось 3 и более пользователей.

Сколько строк останется в выгрузке Яндекс.Метрики после этих действий?

In [70]:
data['data']

[{'dimensions': [{'name': '2018-03-12'},
   {'icon_id': 'desktop',
    'icon_type': 'device',
    'id': 'desktop',
    'name': 'ПК'},
   {'icon_id': None, 'icon_type': None, 'id': None, 'name': None},
   {'icon_id': None, 'icon_type': None, 'name': None}],
  'metrics': [157.0]},
 {'dimensions': [{'name': '2018-03-13'},
   {'icon_id': 'desktop',
    'icon_type': 'device',
    'id': 'desktop',
    'name': 'ПК'},
   {'icon_id': None, 'icon_type': None, 'id': None, 'name': None},
   {'icon_id': None, 'icon_type': None, 'name': None}],
  'metrics': [114.0]},
 {'dimensions': [{'name': '2018-03-02'},
   {'icon_id': 'desktop',
    'icon_type': 'device',
    'id': 'desktop',
    'name': 'ПК'},
   {'icon_id': None, 'icon_type': None, 'id': None, 'name': None},
   {'icon_id': None, 'icon_type': None, 'name': None}],
  'metrics': [111.0]},
 {'dimensions': [{'name': '2018-03-22'},
   {'icon_id': 'desktop',
    'icon_type': 'device',
    'id': 'desktop',
    'name': 'ПК'},
   {'icon_id': None, 'icon

In [124]:
n_rows=100

offset=1

arpu_yandex=[] #дата ,девайс категория , фирма , модель

Started = True

while (len(data['data']) > 0 or Started):
    
    data=report_for_offset_value(offset,n_rows)
    
    Started = False
        
    for line in data['data']:
        line['metrics'] # пользователи
        category=[]
        for item in line['dimensions']:
            category.append(item['name'])
        arpu_yandex.append(category+line['metrics'])
            
        #print(тестовый)
        
    offset+=n_rows
arpu_yandex

[['2018-03-12', 'ПК', None, None, 157.0],
 ['2018-03-13', 'ПК', None, None, 114.0],
 ['2018-03-02', 'ПК', None, None, 111.0],
 ['2018-03-22', 'ПК', None, None, 109.0],
 ['2018-03-19', 'ПК', None, None, 108.0],
 ['2018-03-01', 'ПК', None, None, 106.0],
 ['2018-03-15', 'ПК', None, None, 103.0],
 ['2018-03-14', 'ПК', None, None, 102.0],
 ['2018-03-06', 'ПК', None, None, 97.0],
 ['2018-03-03', 'ПК', None, None, 95.0],
 ['2018-03-26', 'ПК', None, None, 95.0],
 ['2018-03-20', 'ПК', None, None, 94.0],
 ['2018-03-11', 'ПК', None, None, 90.0],
 ['2018-03-21', 'ПК', None, None, 90.0],
 ['2018-03-05', 'ПК', None, None, 86.0],
 ['2018-03-16', 'ПК', None, None, 86.0],
 ['2018-03-04', 'ПК', None, None, 82.0],
 ['2018-03-09', 'ПК', None, None, 82.0],
 ['2018-03-28', 'ПК', None, None, 80.0],
 ['2018-03-07', 'ПК', None, None, 77.0],
 ['2018-03-25', 'ПК', None, None, 77.0],
 ['2018-03-27', 'ПК', None, None, 77.0],
 ['2018-03-29', 'ПК', None, None, 75.0],
 ['2018-03-30', 'ПК', None, None, 75.0],
 ['2018-

In [154]:
len(arpu_yandex)

2414

# Пример частный случай чтобы понять с какитх строк брать

In [81]:
data['data'][87]

{'dimensions': [{'name': '2018-03-04'},
  {'icon_id': 'mobile',
   'icon_type': 'device',
   'id': 'mobile',
   'name': 'Смартфоны'},
  {'icon_id': '7', 'icon_type': 'mobile', 'id': '7', 'name': 'Samsung'},
  {'icon_id': '7', 'icon_type': 'mobile', 'name': 'Galaxy J3'}],
 'metrics': [5.0]}

In [91]:
data['data'][87]['dimensions']

[{'name': '2018-03-04'},
 {'icon_id': 'mobile',
  'icon_type': 'device',
  'id': 'mobile',
  'name': 'Смартфоны'},
 {'icon_id': '7', 'icon_type': 'mobile', 'id': '7', 'name': 'Samsung'},
 {'icon_id': '7', 'icon_type': 'mobile', 'name': 'Galaxy J3'}]

In [98]:
#берем ииз отчета дату
data['data'][87]['dimensions'][0]['name']


'2018-03-04'

In [99]:
#берем ииз отчета deviceCategory
data['data'][87]['dimensions'][1]['name']

'Смартфоны'

In [100]:
#берем ииз отчета mobilePhone
data['data'][87]['dimensions'][2]['name']

'Samsung'

In [ ]:
#берем ииз отчета mobilePhoneModel

In [103]:
data['data'][87]['dimensions'][3]['name']

'Galaxy J3'

In [118]:
arpu_yandex=[]
for item in data['data'][87]['dimensions']:
    #print (item['name'])
    arpu_yandex.append(item['name'])
arpu_yandex

['2018-03-04', 'Смартфоны', 'Samsung', 'Galaxy J3']

# end

In [188]:
columns=['date','deviceCategory','mobilePhone','mobilePhoneModel','users']
dg=pd.DataFrame(arpu_yandex,columns=columns)
dg

,date,deviceCategory,mobilePhone,mobilePhoneModel,users
0,2018-03-12,ПК,None,None,157.0
1,2018-03-13,ПК,None,None,114.0
2,2018-03-02,ПК,None,None,111.0
3,2018-03-22,ПК,None,None,109.0
4,2018-03-19,ПК,None,None,108.0
5,2018-03-01,ПК,None,None,106.0
6,2018-03-15,ПК,None,None,103.0
7,2018-03-14,ПК,None,None,102.0
8,2018-03-06,ПК,None,None,97.0
9,2018-03-03,ПК,None,None,95.0


In [202]:
dg.loc[0]

date                2018-03-12
deviceCategory              ПК
mobilePhone            unnamed
mobilePhoneModel       unnamed
users                      157
Name: 0, dtype: object

In [203]:
dg.loc[0][2] is None

False

In [198]:
dg=dg.fillna('unnamed')

In [204]:
dg_sorted=dg.groupby(['date','deviceCategory','mobilePhone','mobilePhoneModel']).sum()
dg_sorted

users
date       deviceCategory mobilePhone mobilePhoneModel              
2018-03-01 ПК             unnamed     unnamed                  106.0
           Планшеты       Apple       iPad                       2.0
                          CUBE        Mytab-U55GT                1.0
                          Explay      sQuad 7.82 3G              1.0
           Смартфоны      ASUS        ASUS_X008D                 2.0
                                      ASUS_Z008D                 1.0
                                      ASUS_Z00ED                 1.0
                                      ASUS_Z00RD                 1.0
                                      ASUS_Z00YD                 1.0
                                      ZB501KL                    1.0
                                      ZC554KL                    1.0
                          Alcatel     One Touch POP Up           1.0
                          Apple       iPhone                    48.0
                          BQ          BQ-5058                    1.0
                                      BQru-5059                  1.0
                                      BQru-5510                  1.0
                          DEXP        Ixion X150                 1.0
                          Explay      Vega                       1.0
                          Fly         FS509                      1.0
                          HTC         HTC Desire 530             1.0
                                      HTC Desire 830 dual sim    1.0
                          Highscreen  PowerFour                  1.0
                                      Zera F                     1.0
                          Huawei      CAM-L21                    2.0
                                      DLI-TL20                   1.0
                                      EVA-L19                    1.0
                                      FRD-L19                    1.0
                                      HUAWEI CUN-L21             1.0
                                      LYO-L21                    1.0
                                      MYA-L22                    1.0
...                                                              ...
2018-03-31 Смартфоны      Meizu       MZ-M5s                     1.0
                                      MZ-m3 note                 1.0
                          Micromax    Micromax D303              1.0
                                      Micromax Q409              1.0
                                      Micromax Q415              1.0
                          Nokia       Lumia 550                  1.0
                          Samsung     Galaxy A3                  4.0
                                      Galaxy A5                  3.0
                                      Galaxy Alpha               1.0
                                      Galaxy Core 2              1.0
                                      Galaxy J1 Mini             2.0
                                      Galaxy J3                  2.0
                                      Galaxy J5                  1.0
                                      Galaxy J7                  1.0
                                      Galaxy S3 Neo              1.0
                                      Galaxy S5                  1.0
                                      Galaxy S5 Duos             1.0
                                      Galaxy S6 Edge             2.0
                                      Galaxy S7                  2.0
                                      Galaxy S8                  1.0
                                      SM-N950U1                  1.0
                          Sony        Xperia Z3+                 1.0
                          Xiaomi      MI MAX                     1.0
                                      Redmi 3S                   1.0
                                      Redmi 4                    1.0
                                      Redmi 4X               

In [209]:
dg_filter = dg_sorted[dg_sorted['users'] >= 3]
dg_filter

users
date       deviceCategory mobilePhone mobilePhoneModel       
2018-03-01 ПК             unnamed     unnamed           106.0
           Смартфоны      Apple       iPhone             48.0
                          Samsung     Galaxy A3           8.0
                                      Galaxy J1           3.0
                                      Galaxy J3           6.0
                          Xiaomi      Redmi 3S            3.0
                                      Redmi 4X            3.0
2018-03-02 ПК             unnamed     unnamed           111.0
           Планшеты       Apple       iPad                3.0
           Смартфоны      Apple       iPhone             62.0
                          Samsung     Galaxy A3           4.0
                                      Galaxy A5           5.0
                                      Galaxy J3           5.0
                          Xiaomi      Redmi 4X            6.0
                                      Redmi Note 4        7.0
2018-03-03 ПК             unnamed     unnamed            95.0
           Планшеты       Apple       iPad                4.0
                          Samsung     Galaxy Tab 4 7.0    3.0
           Смартфоны      Apple       iPhone             55.0
                          Huawei      Honor 4C            3.0
                          Samsung     Galaxy A3           3.0
                                      Galaxy A5           3.0
                                      Galaxy J1           4.0
                                      Galaxy J3           4.0
                                      Galaxy J5           4.0
                                      Galaxy S7           3.0
                          Xiaomi      Redmi 3S            3.0
                                      Redmi 4X            5.0
                                      Redmi Note 4        6.0
2018-03-04 ПК             unnamed     unnamed            82.0
...                                                       ...
2018-03-26 Смартфоны      Samsung     Galaxy J7           4.0
                          unnamed     unnamed             3.0
2018-03-27 ПК             unnamed     unnamed            77.0
           Смартфоны      Apple       iPhone             29.0
                          Samsung     Galaxy A3           3.0
                                      Galaxy J1           3.0
                                      Galaxy S7           3.0
                          Xiaomi      Redmi Note 4        3.0
2018-03-28 ПК             unnamed     unnamed            80.0
           Планшеты       Apple       iPad                3.0
           Смартфоны      Apple       iPhone             28.0
                          Samsung     Galaxy S7           4.0
                          Xiaomi      Redmi Note 4        3.0
2018-03-29 ПК             unnamed     unnamed            75.0
           Планшеты       Apple       iPad                3.0
           Смартфоны      Apple       iPhone             22.0
                          HTC         HTC Desire 628      3.0
                          Samsung     Galaxy A5           4.0
2018-03-30 ПК             unnamed     unnamed            75.0
           Смартфоны      Apple       iPhone             23.0
                          Samsung     Galaxy A5           3.0
                                      Galaxy J3           3.0
                          Xiaomi      Redmi 3S            6.0
                                      Redmi 4X            5.0
                                      Redmi Note 4        4.0
2018-03-31 ПК             unnamed     unnamed            53.0
           Смартфоны      Apple       iPhone             35.0
                          Samsung     Galaxy A3           4.0
                                      Galaxy A5           3.0
                          Xiaomi      Redmi 4X            5.0

[248 rows x 1 columns]

In [210]:
len(dg_filter)

248

In [ ]:
# должно быть 242

# Проверка 4
(1 возможный балл)
После совмещения выгрузок Яндекс.Метрики и файла с выручкой надо посчитать значение ARPU для каждой строки получившегося датафрейма. Можно использовать метод apply + отдельную функцию. Или использовать метод apply с lambda-функцией. Обратите внимание, что столбец 'revenue' не всегда автоматически распознается как тип float.

Какое значение ARPU получится у строки с категорией устройства 'ПК' (модель и тип устройства при этом будут не определены)? Ответ округлите до двух знаков после запятой. Формат ответа: 0.84

In [261]:
df[df['deviceCategory']!='Смартфоны'][df['deviceCategory']!='Планшеты']

D:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,mobilePhone,mobilePhoneModel,deviceCategory,revenue
465,None,None,ПК,2667.568827
468,None,None,ТВ,1.299006


In [302]:
dg_4proverka=dg_filter.reset_index()
dg_4proverka.head()

,date,deviceCategory,mobilePhone,mobilePhoneModel,users
0,2018-03-01,ПК,unnamed,unnamed,106.0
1,2018-03-01,Смартфоны,Apple,iPhone,48.0
2,2018-03-01,Смартфоны,Samsung,Galaxy A3,8.0
3,2018-03-01,Смартфоны,Samsung,Galaxy J1,3.0
4,2018-03-01,Смартфоны,Samsung,Galaxy J3,6.0


In [269]:
dg_4proverka[dg_4proverka['deviceCategory']=='ПК']['users'].sum()

2715.0

In [270]:
#ПК users = 106 and revenue = 2667.568827
ans=2667.568827/2715.0
ans

0.9825299546961326

# Итоговый результат экзамена
(1 возможный балл)
Наконец, сортируем получившийся датафрейм по убыванию столбца с вычисленным значением ARPU. И проверяем какая модель устройства mobilePhoneModel имеет максимальный ARPU:

In [ ]:
# вначале сумируем по user м отсавляем толко mobilePhoneModel

In [304]:
dg5=dg_4proverka.groupby(['mobilePhoneModel']).sum()
dg5.tail()

,users
mobilePhoneModel,
Redmi Note 5A,3.0
WAS-LX1,3.0
iPad,76.0
iPhone,1282.0
unnamed,2718.0


In [291]:
dg5=dg5.reset_index()

In [292]:
dh=dg5.merge(df, how = 'left', on = 'mobilePhoneModel')
dh.head()

,mobilePhoneModel,users,mobilePhone,deviceCategory,revenue
0,ASUS_X008D,3.0,ASUS,Смартфоны,11.341971
1,ASUS_Z00AD,3.0,ASUS,Смартфоны,6.021662
2,DIG-L21HN,3.0,Huawei,Смартфоны,6.227160
3,Galaxy A3,80.0,Samsung,Смартфоны,66.939521
4,Galaxy A5,64.0,Samsung,Смартфоны,64.093878


In [293]:
dh['ARPU']=dh['revenue']/dh['users']

In [294]:
dh_sorted=dh.sort_values(['ARPU'])

In [297]:
dh_sorted.tail()

,mobilePhoneModel,users,mobilePhone,deviceCategory,revenue,ARPU
12,Galaxy S4,4.0,Samsung,Смартфоны,14.642638,3.660660
0,ASUS_X008D,3.0,ASUS,Смартфоны,11.341971,3.780657
22,MI 5,3.0,Xiaomi,Смартфоны,14.629858,4.876619
26,Redmi 4,3.0,Xiaomi,Смартфоны,16.244476,5.414825
35,unnamed,2718.0,NaN,NaN,NaN,NaN


# Для всех а не >=3

In [306]:
dg_sorted

users
date       deviceCategory mobilePhone mobilePhoneModel              
2018-03-01 ПК             unnamed     unnamed                  106.0
           Планшеты       Apple       iPad                       2.0
                          CUBE        Mytab-U55GT                1.0
                          Explay      sQuad 7.82 3G              1.0
           Смартфоны      ASUS        ASUS_X008D                 2.0
                                      ASUS_Z008D                 1.0
                                      ASUS_Z00ED                 1.0
                                      ASUS_Z00RD                 1.0
                                      ASUS_Z00YD                 1.0
                                      ZB501KL                    1.0
                                      ZC554KL                    1.0
                          Alcatel     One Touch POP Up           1.0
                          Apple       iPhone                    48.0
                          BQ          BQ-5058                    1.0
                                      BQru-5059                  1.0
                                      BQru-5510                  1.0
                          DEXP        Ixion X150                 1.0
                          Explay      Vega                       1.0
                          Fly         FS509                      1.0
                          HTC         HTC Desire 530             1.0
                                      HTC Desire 830 dual sim    1.0
                          Highscreen  PowerFour                  1.0
                                      Zera F                     1.0
                          Huawei      CAM-L21                    2.0
                                      DLI-TL20                   1.0
                                      EVA-L19                    1.0
                                      FRD-L19                    1.0
                                      HUAWEI CUN-L21             1.0
                                      LYO-L21                    1.0
                                      MYA-L22                    1.0
...                                                              ...
2018-03-31 Смартфоны      Meizu       MZ-M5s                     1.0
                                      MZ-m3 note                 1.0
                          Micromax    Micromax D303              1.0
                                      Micromax Q409              1.0
                                      Micromax Q415              1.0
                          Nokia       Lumia 550                  1.0
                          Samsung     Galaxy A3                  4.0
                                      Galaxy A5                  3.0
                                      Galaxy Alpha               1.0
                                      Galaxy Core 2              1.0
                                      Galaxy J1 Mini             2.0
                                      Galaxy J3                  2.0
                                      Galaxy J5                  1.0
                                      Galaxy J7                  1.0
                                      Galaxy S3 Neo              1.0
                                      Galaxy S5                  1.0
                                      Galaxy S5 Duos             1.0
                                      Galaxy S6 Edge             2.0
                                      Galaxy S7                  2.0
                                      Galaxy S8                  1.0
                                      SM-N950U1                  1.0
                          Sony        Xperia Z3+                 1.0
                          Xiaomi      MI MAX                     1.0
                                      Redmi 3S                   1.0
                                      Redmi 4                    1.0
                                      Redmi 4X               

In [307]:
dg5=dg.groupby(['mobilePhoneModel']).sum()

In [314]:
dg5=dg5.reset_index()

In [315]:
dh=dg5.merge(df, how = 'left', on = 'mobilePhoneModel')
dh=dh.dropna()
dh

,mobilePhoneModel,users,mobilePhone,deviceCategory,revenue
0,2014817,1.0,Xiaomi,Смартфоны,0.794720
1,4013D,2.0,Alcatel,Смартфоны,1.144198
2,4027D,3.0,Alcatel,Смартфоны,1.895412
3,4032D,2.0,Alcatel,Смартфоны,1.024806
4,5010D,1.0,Alcatel,Смартфоны,0.486240
5,5015D,2.0,Alcatel,Смартфоны,1.361705
6,5015X,1.0,Alcatel,Смартфоны,0.683021
7,5017X,3.0,Alcatel,Смартфоны,1.638464
8,5019D,1.0,Alcatel,Смартфоны,0.804883
9,5045D,1.0,Alcatel,Смартфоны,0.712779


In [316]:
dh['ARPU']=dh['revenue']/dh['users']

In [317]:
dh_sorted=dh.sort_values(['ARPU'])

In [318]:
dh_sorted

,mobilePhoneModel,users,mobilePhone,deviceCategory,revenue,ARPU
205,Galaxy S8,26.0,None,Смартфоны,0.352001,0.013539
392,Lumia 640,6.0,Nokia,Планшеты,0.308920,0.051487
390,Lumia 630,5.0,Nokia,Планшеты,0.396348,0.079270
612,Tab_2,5.0,Beeline,Планшеты,0.523609,0.104722
57,BND-L21,6.0,Samsung,Смартфоны,0.643892,0.107315
454,Micromax E313,1.0,Micromax,Смартфоны,0.285905,0.285905
457,Micromax Q334,2.0,Micromax,Смартфоны,0.574928,0.287464
560,Rover,1.0,Rover,Планшеты,0.290924,0.290924
438,MZ-U10,1.0,Meizu,Смартфоны,0.299989,0.299989
452,Micromax D303,1.0,Micromax,Смартфоны,0.306307,0.306307
